In [1]:
import sys
sys.path.insert(0, "../..")
def set_size(width = "thesis", fraction=1, subplots=(1, 1)):
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('seaborn')

from os import listdir
from os.path import isfile, join

from module.base.network import Network
from module.simulation.meanfield import MeanField
import module.components.CONST as CONST
from module.simulation.set_meanfield2 import SetMeanField2
from module.simulation.masterequation import MasterEquation
from module.simulation.quick_meanfield2 import QuickMeanField2
from module.simulation.meanfield2 import MeanField2
from module.components.lawrence_dist import LawrenceDist
from module.components.discrete_gaussian1D import DiscreteGaussian1D

from module.components.restricted_gaussian1D import RestrictedGaussian1D
from module.components.p2_4dist import p2_4dist

/tmp/ipykernel_8716/2397363.py:30: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [9]:
voltages = np.loadtxt("../../data/cont_comparison/voltage_configs.csv")

In [41]:
kmc_data = np.loadtxt("../../data/precision_currents/Nx=4_Ny=4_Nz=1_Ne=4.csv", delimiter=",", skiprows=1)
kmc_currents = kmc_data[:, 7] * CONST.electron_charge / 1000

In [61]:
mf_currents = []

for i in range(200):
    net = Network(4,4,1,[[0,0,0],[3,0,0],[0,3,0],[3,3,0]])
    net.set_voltage_config(voltages[i], 0)
    mf = MeanField(net)
    mean = mf.numeric_integration_solve(verbose = True, N = 50)
    mf_currents.append(-mf.calc_expected_electrode_rates(mean, 3) * CONST.electron_charge)

mf_currents = np.array(mf_currents)

convergence: 0.0031580597200012023
convergence: 0.0032091071687270906
convergence: 0.0032651767891341416
convergence: 0.0033249051339031693
convergence: 0.0033870976903149635
convergence: 0.003451244723538034
convergence: 0.003517230035030648
convergence: 0.0035830573127744736
convergence: 0.003650376965712465
convergence: 0.0037164414200167514
convergence: 0.0037826976221305084
convergence: 0.003847419260220794
convergence: 0.0039101077543867435
convergence: 0.0039704182562476376
convergence: 0.004027540130138084
convergence: 0.00408102135847762
convergence: 0.0041318046535898795
convergence: 0.0041774608406195055
convergence: 0.004217013076679245
convergence: 0.004250910388621826
convergence: 0.004278628607615818
convergence: 0.004299038990608482
convergence: 0.004310079884563819
convergence: 0.00431123784499262
convergence: 0.004302215532628981
convergence: 0.004300678746025202
convergence: 0.004797014142595417
convergence: 0.004165272741372562
convergence: 0.004289331988686293
conv

In [32]:
qmf2_currents = []

In [33]:
for i in range(200):
    net = Network(4,4,1,[[0,0,0],[3,0,0],[0,3,0],[3,3,0]])
    net.set_voltage_config(voltages[i], 0)
    qmf2 = QuickMeanField2(net)
    qmf2.ADAM_solve(verbose = True, N = 300)
    qmf2_currents.append(-qmf2.calc_expected_electrode_current(3))

ADAM convergence: (0.011207590034048776, 0.03899986217831519)
ADAM convergence: (0.017273300679160223, 0.05653201997997563)
ADAM convergence: (0.01736954730532817, 0.059057426522304585)
ADAM convergence: (0.0086615670563411, 0.02785074924729887)
ADAM convergence: (0.01276042526563451, 0.04325567447027918)
ADAM convergence: (0.02237371134214294, 0.07563462073410572)
ADAM convergence: (0.02402077072657972, 0.07384743061580787)
ADAM convergence: (0.019007113991451816, 0.06020235887075706)
ADAM convergence: (0.026629219476423552, 0.0786920798752338)
ADAM convergence: (0.014906278262828584, 0.04554946776693407)
ADAM convergence: (0.021963925000697136, 0.07479047951554674)
ADAM convergence: (0.013547329534348933, 0.02289875396118642)
ADAM convergence: (0.01717186865056399, 0.04639356617282219)
ADAM convergence: (0.02208520309524237, 0.06387266949670686)
ADAM convergence: (0.02104056990114161, 0.05177350916519344)
ADAM convergence: (0.012505349199245624, 0.062490836231746616)
ADAM convergence

In [35]:
qmf2_currents = np.array(qmf2_currents)

In [46]:
rel_errs = np.abs((kmc_currents - qmf2_currents)/kmc_currents)
valid = np.where(rel_errs < 0.2)[0]

In [54]:
print("High Precision 4x4 currents QMF2 with RE =", np.mean(rel_errs[valid]) * 100, "%")

High Precision 4x4 curretns with RE = 3.125016207614269 %


In [65]:
rel_errs = np.abs((kmc_currents - mf_currents)/kmc_currents)
valid = np.where(rel_errs < 0.3)[0]

In [67]:
print("High Precision 4x4 currents MF1 with RE =", np.mean(rel_errs[valid]) * 100, "%")

High Precision 4x4 currents MF1 with RE = 9.812944037098994 %
